In [ ]:
%matplotlib widget
  
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from random import randrange
from threading import Thread
import time
import sys
import numpy as np
  

def  impuls(n):
    if n==0:
        return 1
    return 0

def shift(x, old_x):
    return np.concatenate(([x],old_x[0:-1]))

def system1(x_n,old_x,delay = 4):
    return 1/(delay+1) * (x_n + np.sum(old_x[0:delay]))
    

## System 1
The system we have implemented is 
$$ 
\begin{align}
y[n]=\sum_{k=n-N}^{n} \frac{x[k]}{N+1}
\end{align}
$$
with $N=4$

In [ ]:

class LiveGraph:
    def __init__(self,func):
        self.func = func
        self.n_data, self.y_data = [], []
        self.d_data = []
        self.old_x = np.zeros(50)
        self.figure = plt.figure()
        self.ax1 = plt.subplot(211)
        self.ax2 = plt.subplot(212)

        self.ax1.set_ylabel('x[n]')
        self.ax2.set_ylabel('y[n]')
        self.ax2.set_xlabel('n')

        self.line1, = self.ax1.plot(self.n_data, self.d_data,'o',markerfacecolor='red')
        self.line2, = self.ax2.plot(self.n_data, self.y_data,'o',markerfacecolor='red')
        plt.grid()
        self.interval = 500
        self.animation = FuncAnimation(self.figure, self.update, interval=self.interval, save_count=10)
        self.th = Thread(target=self.thread_f, daemon=True)
        self.th.start()


    def update(self, frame):
        self.line1.set_data(self.n_data, self.d_data)
        self.line2.set_data(self.n_data, self.y_data)
        self.ax1.relim()
        self.ax2.relim()
        self.ax1.autoscale_view()
        self.ax2.autoscale_view()
        return self.line1

    def show(self):
        plt.show()

    def thread_f(self):
        n = -10
        while n<40:
            x_n = impuls(n)
            self.n_data.append(n)
            self.d_data.append(x_n)
            self.y_data.append(self.func(x_n,self.old_x))   
            if len(self.n_data)>20:
                self.y_data.pop(0)
                self.d_data.pop(0)
                self.n_data.pop(0)

            self.old_x=shift(x_n, self.old_x)
            n += 1
            time.sleep(self.interval/1000)  
        sys.exit()
          


In [ ]:
g = LiveGraph(system1)
g.show()

## System 2
The system we have implemented is 
$$ 
\begin{align}
h[k]&=[1,2,3,2,1]\\
    y[n]&=h[k]* x[n]
\end{align}
$$

In [ ]:
def system2(x_n,old_x):
    h_k = [1,2,3,2,1]
    
    Xs = np.concatenate(([x_n],old_x))

    conv = np.convolve(h_k,Xs)[4]
    return conv

g = LiveGraph(system2)
g.show()
